<a href="https://colab.research.google.com/github/Holy-Morphism/GenAI/blob/main/MultiModalRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Model RAG

In [ ]:
%pip install -q llama-index-llms-gemini
%pip install -q llama-index-embeddings-gemini
%pip install -q llama-index-embeddings-huggingface
%pip install -q llama-index-vector-stores-chroma
%pip install -q llama-index-postprocessor-cohere-rerank
%pip install -q llama-index
%pip install -q pyvis
%pip install -q -U openai-whisper
%pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




## Import Libraries

Various libraries are imported to handle environment variables, LLM settings, embedding models, chunking, vector embeddings, and storage contexts.



In [ ]:
import os

from llama_index.llms.gemini import Gemini
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext



## Get Environment Variables

Environment variables are loaded using the [`dotenv`](command:_github.copilot.openSymbolFromReferences?%5B%22dotenv%22%2C%5B%7B%22uri%22%3A%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22f%3A%5C%5CAI%5C%5Cnotebooks%5C%5CPLPS%5C%5CAdvanceRAG.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Ff%253A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22path%22%3A%22%2Ff%3A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22scheme%22%3A%22file%22%7D%2C%22pos%22%3A%7B%22line%22%3A36%2C%22character%22%3A10%7D%7D%5D%5D "Go to definition") library.



In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
COHERERERANK_API_KEY =  userdata.get("COHERERERANK_API_KEY")



## Set LLM

The LLM (Language Learning Model) is set using the [`Gemini`](command:_github.copilot.openSymbolFromReferences?%5B%22Gemini%22%2C%5B%7B%22uri%22%3A%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22f%3A%5C%5CAI%5C%5Cnotebooks%5C%5CPLPS%5C%5CAdvanceRAG.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Ff%253A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22path%22%3A%22%2Ff%3A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22scheme%22%3A%22file%22%7D%2C%22pos%22%3A%7B%22line%22%3A38%2C%22character%22%3A11%7D%7D%5D%5D "Go to definition") model.



In [ ]:
Settings.llm = llm = Gemini(model="models/gemini-1.5-flash",api_key = GOOGLE_API_KEY)



## Set Embedding Model

The embedding model is set using the [`GeminiEmbedding`](command:_github.copilot.openSymbolFromReferences?%5B%22GeminiEmbedding%22%2C%5B%7B%22uri%22%3A%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22f%3A%5C%5CAI%5C%5Cnotebooks%5C%5CPLPS%5C%5CAdvanceRAG.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Ff%253A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22path%22%3A%22%2Ff%3A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22scheme%22%3A%22file%22%7D%2C%22pos%22%3A%7B%22line%22%3A110%2C%22character%22%3A47%7D%7D%5D%5D "Go to definition") class.



In [ ]:
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"
Settings.embed_model = em = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY, title="These are research papers"
)



## Load Data

Data is loaded from a specified directory using the [`SimpleDirectoryReader`](command:_github.copilot.openSymbolFromReferences?%5B%22SimpleDirectoryReader%22%2C%5B%7B%22uri%22%3A%7B%22%24mid%22%3A1%2C%22fsPath%22%3A%22f%3A%5C%5CAI%5C%5Cnotebooks%5C%5CPLPS%5C%5CAdvanceRAG.ipynb%22%2C%22_sep%22%3A1%2C%22external%22%3A%22file%3A%2F%2F%2Ff%253A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22path%22%3A%22%2Ff%3A%2FAI%2Fnotebooks%2FPLPS%2FAdvanceRAG.ipynb%22%2C%22scheme%22%3A%22file%22%7D%2C%22pos%22%3A%7B%22line%22%3A51%2C%22character%22%3A52%7D%7D%5D%5D "Go to definition").



In [ ]:
from llama_index.core import  SimpleDirectoryReader

documents = SimpleDirectoryReader("drive/MyDrive/data").load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:04<00:00,  4.78s/file]




## Ingestion Pipeline




In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
# import asyncio
# loop = asyncio.new_event_loop()
# asyncio.set_event_loop(loop)

# db = chromadb.PersistentClient(path="./chroma_db")
# chroma_collection = db.get_or_create_collection("research_papers")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# pipeline = IngestionPipeline(
#     transformations=[
#         SentenceSplitter(chunk_size=2000, chunk_overlap=200),
#         em,
#     ],
#     vector_store=vector_store
# )

# pipeline.run(documents=documents)
# index = VectorStoreIndex.from_vector_store(vector_store, embed_model=em,show_progress=True)

text_splitter = SentenceSplitter(chunk_size=2000, chunk_overlap=200)

Settings.text_splitter = text_splitter

db = chromadb.PersistentClient(path="./chroma_db")

# get collection
chroma_collection = db.get_or_create_collection("research_papers")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter], storage_context=storage_context,show_progress=True
)

Parsing nodes:   0%|          | 0/48 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/49 [00:00<?, ?it/s]

## Query Pipeline

### Setting the retriver

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
# Tried to use other retrievers but due to there argumnets I had to change the whole previous logic
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,

    embed_model=em
)

### Setting the ReRanker

In [ ]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

reranker = CohereRerank(api_key=COHERERERANK_API_KEY, top_n=3)

### Setting the Reponse Synthesizer

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize

response_synthesizer = TreeSummarize(
   llm=llm
)

### Putting it all together

In [ ]:
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

prompt_str = "Please answer this question: {question}"
prompt_tmpl = PromptTemplate(prompt_str)

pipeline = QueryPipeline()

pipeline.add_modules(
    {
        "llm": llm,
        "prompt_tmpl": prompt_tmpl,
        "retriever": retriever,
        "synthesizer": response_synthesizer,
        "reranker": reranker,
    }
)

pipeline.add_link("prompt_tmpl", "llm")
pipeline.add_link("llm", "retriever")
pipeline.add_link("retriever", "reranker", dest_key="nodes")
pipeline.add_link("llm", "reranker",dest_key='query_str')
pipeline.add_link("reranker", "synthesizer", dest_key="nodes")
pipeline.add_link("llm", "synthesizer",dest_key='query_str')


### Running the pipeline

In [ ]:
from IPython.display import display, Markdown

output = pipeline.run(question="What are KAN networks ?")

display(Markdown(str(output)))


KAN networks are a type of neural network that uses learnable activation functions. They are similar to Multilayer Perceptrons (MLPs) but with the key difference that the activation functions are not fixed but rather learned during training. This allows KANs to learn more complex and expressive functions than traditional MLPs. 


In [ ]:
from IPython.display import display, Markdown

def run_pipeline(prompt):
    output = pipeline.run(question=prompt)
    return str(output)


In [ ]:
import whisper

model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 124MiB/s]


In [ ]:
def transcribe_audio_pipeline(audio):
  # load audio and pad/trim it to fit 30 seconds
  audio = whisper.load_audio(audio)
  audio = whisper.pad_or_trim(audio)

  # make log-Mel spectrogram and move to the same device as the model
  # Use the correct number of mel filters (n_mels)
  mel = whisper.log_mel_spectrogram(audio, n_mels=80).to(model.device)

  # detect the spoken language
  _, probs = model.detect_language(mel)
  #print(f"Detected language: {max(probs, key=probs.get)}")

  # decode the audio
  options = whisper.DecodingOptions()
  result = whisper.decode(model, mel, options)

  # print the recognized text
  rp = result.text
  print(rp)
  query = run_pipeline(rp)
  print(query)

  return query

In [ ]:
transcribe_audio_pipeline('drive/MyDrive/voice/test.mp3')


What is our coal migorow of Arnold Networks?
The provided text discusses Kolmogorov-Arnold Networks (KANs) and their potential as an alternative to Multi-Layer Perceptrons (MLPs).  The term "coal migorow" is not mentioned in the text.  Please clarify what you mean by "coal migorow" so I can assist you further. 



'The provided text discusses Kolmogorov-Arnold Networks (KANs) and their potential as an alternative to Multi-Layer Perceptrons (MLPs).  The term "coal migorow" is not mentioned in the text.  Please clarify what you mean by "coal migorow" so I can assist you further. \n'

In [ ]:
def random_response(message, history):
    return run_pipeline(message)

In [ ]:
import gradio as gr

demo = gr.Interface(fn=transcribe_audio_pipeline, inputs="audio", outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://67b38e28bf9f66aa88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
